In [1]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm

def set_cliff_threshold(value):
    def modify_parameters(parameters):
        parameters.simulation.cliff_threshold.update(period="year:2020:10", value=value)
        return parameters
    class reform(Reform):
        def apply(self):
            self.modify_parameters(modify_parameters)
    return reform

sim = Microsimulation()
CLIFF_THRESHOLDS = [0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]
cliff_rates = []
cliff_gaps = []

for cliff_threshold in tqdm(CLIFF_THRESHOLDS):
    sim = Microsimulation(reform=set_cliff_threshold(cliff_threshold))
    cliff_rates.append(sim.calc("is_on_cliff")[sim.calc("is_adult")].mean())
    cliff_gaps.append(sim.calc("cliff_gap").sum())

df = pd.DataFrame({"cliff_threshold": CLIFF_THRESHOLDS, "cliff_rate": cliff_rates, "cliff_gap": cliff_gaps})


  0%|          | 0/11 [00:00<?, ?it/s]

Increasing employment income by 0 for 152732 people


  9%|▉         | 1/11 [00:14<02:23, 14.36s/it]

Increasing employment income by 1000 for 152732 people


 18%|█▊        | 2/11 [00:28<02:08, 14.25s/it]

...

 82%|████████▏ | 9/11 [02:04<00:27, 13.76s/it]

Increasing employment income by 9000 for 152732 people


 91%|█████████ | 10/11 [02:18<00:13, 13.75s/it]

Increasing employment income by 10000 for 152732 people


100%|██████████| 11/11 [02:32<00:00, 13.83s/it]


In [5]:
import plotly.express as px

px.bar(
    df,
    x="cliff_threshold",
    y="cliff_rate",
    title="Cliff rate by cliff threshold (pooled)",
    text="cliff_rate",
).update_layout(
    xaxis_title="Cliff threshold",
    yaxis_title="Cliff rate",
    xaxis_tickformat="$,.0f",
    yaxis_tickformat=".1%",
    xaxis_tickvals=CLIFF_THRESHOLDS,
    height=500,
    width=800,
).update_traces(texttemplate="%{text:.1%}", textposition="inside")

In [6]:
import plotly.express as px

df["gap_in_billions"] = df["cliff_gap"] / 1e9

px.bar(
    df,
    x="cliff_threshold",
    y="gap_in_billions",
    title="Cliff gap by cliff threshold (pooled)",
    text="gap_in_billions",
).update_layout(
    xaxis_title="Cliff threshold",
    yaxis_title="Cliff gap",
    xaxis_tickformat="$,.0f",
    yaxis_tickformat="$,.0f",
    yaxis_ticksuffix="bn",
    xaxis_tickvals=CLIFF_THRESHOLDS,
    height=500,
    width=800,
).update_traces(texttemplate="%{text:$,.1f}bn", textposition="inside", textangle=0)